In [624]:
import numpy as np 
import numpy.linalg as npla
import scipy 
from scipy import sparse
import scipy.sparse.linalg as spla
import time as time 

In [625]:
def Jsolve(A, b, tol = 1e-8, max_iters = 1000, callback = None):
    """Solve a linear system Ax = b for x by the Jacobi iterative method.
    Parameters: 
      A: the matrix.
      b: the right-hand side vector.
      tol = 1e-8: the relative residual at which to stop iterating.
      max_iters = 1000: the maximum number of iterations to do. 
      callback = None: a user function to call at every iteration. 
        The callback function has arguments 'x', 'iteration', and 'residual'
    Outputs (in order):
      x: the computed solution
      rel_res: list of relative residual norms at each iteration.
        The number of iterations actually done is len(rel_res) - 1
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"

    # Split A into diagonal D plus off-diagonal C
    d = A.diagonal()         # diagonal elements of A as a vector
    C = A.copy()             # copy of A ...
    C.setdiag(np.zeros(n))   # ... without the diagonal
    
    # Initial guess: x = 0
    x = np.zeros(n)

    # Vector of relative residuals
    # Relative residual is norm(residual)/norm(b)
    # Intitial residual is b - Ax for x=0, or b
    rel_res = [1.0]
        
    # Call user function if specified
    if callback is not None:
        callback(x = x, iteration = 0, residual = 1)

    # Iterate
    for k in range(1, max_iters+1):
        # New x
        x = (b - C @ x) / d

        # Record relative residual (this can be done instead of error)
        # Remember: rel_res = error / some_relative_reference
        this_rel_res = npla.norm(b - A @ x) / npla.norm(b)
        rel_res.append(this_rel_res)
                
        # Call user function if specified
        if callback is not None:
            callback(x = x, iteration = k, residual = this_rel_res)
                        
        # Stop if within tolerance    
        if this_rel_res <= tol:
            break
            
    return (x, rel_res)

In [626]:
def Lsolve(L, b):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
    Output:
      y: the solution vector to L @ y == b
    """
    
    # Check the input
    m, n = L.shape
    assert m == n, "matrix L must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    
    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

In [627]:
def Usolve(U, y):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
    Output:
      x: the solution vector to U @ x == y
    """
    m, n = U.shape
    assert m == n, "matrix U must be square"
    assert np.all(np.triu(U) == U), "matrix U must be upper triangular"
    for i in range(n):
        assert U[i][i] != 0
    
    x = y.astype(np.float64).copy()

    for row in range(n):
        x[n-row-1] = x[n-row-1]  / U[n-row-1, n-row-1] 
        x[:n-row-1] -= x[n-row-1] * U[:n-row-1, n-row-1]
    return x

In [628]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
     
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(np.abs(LU[piv_col:, piv_col]))   # Added np.abs() to fix bug
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            # print("Before:")
            # print(LU)

            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[ [piv_col, piv_row] ]      = p[[piv_row, piv_col]]
            
            # print("After: ")
            # print(LU)
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        
        # This is the standard "core" of the algorithm (same as in LUfactorNoPiv)
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            # print("Did add on row: " + str(row) + "Column: " + str(piv_col))
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    # This will return the L, U, AS WELL AS p (the permutation vector).
    return (L, U, p)

In [629]:
def make_A_3D(k):
    #Revised make_A for a k by k by k grid 
    triples = [] 
    #idea of triples stays the same 
    for a in range(k):
        for b in range(k):
            for c in range(k):
                #row of matrix is the grid point 
                row = c + b * k + a * (k ** 2)
                triples.append((row,row, 6.0))
                #connect to the left grid neighbor
                if c > 0:
                    triples.append((row,row - 1, -1.0))
                #right neighbor
                if c < k - 1:
                    triples.append((row,row + 1, -1.0))
                #neighbor aboove
                if b > 0:
                    triples.append((row,row - k, -1.0))
                #neighbor below
                if b < k -1:
                    triples.append((row,row + k, -1.0))
                #neighbor up
                if a > 0:
                    triples.append((row,row - (k ** 2), -1.0))
                #neighbor down 
                if a < k -1:
                    triples.append((row,row + (k **2), -1.0))
    ndim = k * k * k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values, (rownum, colnum)), shape = (ndim, ndim))            
    return A            

In [630]:
k = 15
A = make_A_3D(k)
print("k:", k)
print("dimensions:", A.shape)
print("nonzeros:", A.size)
# #print(’A as sparse matrix:’); print(A)
# print("A as dense matrix:"); print(A.todense())


k: 15
dimensions: (3375, 3375)
nonzeros: 22275


In [631]:
#Creating arbitrary b 
b = np.array(np.round(np.random.rand(k**3)* 100))
print(b)

[22. 19. 44. ... 25. 12. 60.]


In [632]:
#JSOLVE FROM CLASS 

pre_time = time.time()
x, rel_res = Jsolve(A,b)
post_time = time.time()
print("When k is", k , ", the relative residual is", npla.norm(A@x - b)/npla.norm(b), "and the run time is", post_time - pre_time)
# print("Time taken: ", post_time-pre_time)
# print("Relative residual: " ,rel_res)
# print("Relative residual norm: " , npla.norm(rel_res))

When k is 15 , the relative residual is 9.974348961426036e-09 and the run time is 0.06411361694335938


In [633]:
#SCIPY - spla.cg()
prev_time = time.time()
x, info = spla.cg(A,b)
post_time = time.time()
print("When k is", k , ", the relative residual is", npla.norm(A@x - b)/npla.norm(b), "and the run time is", post_time - pre_time)


When k is 15 , the relative residual is 7.849671864335163e-06 and the run time is 0.13931679725646973


In [634]:
#SCIPY - spla.spsolve()
prev_time = time.time()
x = spla.spsolve(A,b)
post_time = time.time()
print("When k is", k , ", the relative residual is", npla.norm(A@x - b)/npla.norm(b), "and the run time is", post_time - pre_time)



When k is 15 , the relative residual is 1.0451485762556632e-14 and the run time is 0.24664664268493652


In [635]:

prev_time = time.time()
A = np.array(A.todense())
L,U,p = LUfactor(A)
y = Lsolve(L,b[p])
x = Usolve(U,y)
post_time = time.time()
print("When k is", k , ", the relative residual is", npla.norm(A@x - b)/npla.norm(b), "and the run time is", post_time - pre_time)




When k is 15 , the relative residual is 2.855927552870107e-14 and the run time is 31.04134202003479
